# Homework nr. 2 - data cleaning (deadline 8/11/2018)

In short, the main task is to clean The Metropolitan Museum of Art Open Access dataset.
  
> The instructions are not given in details: It is up to you to come up with ideas on how to fulfill the particular tasks as best you can. ;)

## What are you supposed to do:

  1. Download the dataset MetObjects.csv from the repository https://github.com/metmuseum/openaccess/.
  2. Check consistency of at least three features where you expect problems (include "Object Name" feature).
  3. Select some features where you expect integrity problems (describe your choice) and check integrity of those features.
  4. Convert at least five features to a proper data type. Choose at least one numeric, one categorical and one datetime.
  5. Find some outliers (describe your selection criteria).
  6. Detect missing data in at least three features, convert them to a proper representation (if they are already not), and impute missing values in at least one feature.

**If you do all this properly, you will obtain 6 points**

To earn **extra two points** you can do some of these:
  * Focus more precisely on cleaning of the "Medium" feature. Such if you like to use it in KNN based algorithms later.
  * Focus on the extraction of physical dimensions of each item (width, depth and height in centimeters) from the "Dimensions" feature.

## Comments

  * Please follow the instructions from https://courses.fit.cvut.cz/MI-PDD/homeworks/index.html.
  * If the reviewing teacher is not satisfied, he can give you another chance to rework your homework and to obtain more points.

In [ ]:
import pandas as pd
import re
import numpy as np
import datetime
import seaborn as sns

data = pd.read_csv('MetObjects.csv')

## Check consistency of at least three features where you expect problems (include "Object Name" feature).

### Feature: Object Name

In [ ]:
len(list(data['Object Name'].unique()))

In [ ]:
data_edited = data.copy()
data_edited['Object Name'] = data_edited['Object Name'].str.replace('\r\n', ' ')
data_edited['Object Name'] = data_edited['Object Name'].str.replace('\(\?\)', ' ')
data_edited['Object Name'] = data_edited['Object Name'].str.replace('\?', ' ')
data_edited['Object Name'] = data_edited['Object Name'].str.strip()
data_edited['Object Name'] = data_edited['Object Name'].str.lower()


In [ ]:
len(list(data_edited['Object Name'].unique()))

### Feature: Artist Role

In [ ]:
print('Before: '+str(len(data['Artist Role'].unique())))

In [ ]:
def items_to_set(input):
    edited_input = str(input).strip()
    edited_input = edited_input.replace(' and ','|')
    edited_input = edited_input.replace(',','|')
    edited_input = edited_input.lower()
    edited_input = edited_input.strip()
    roles = edited_input.split('|')
    roles_set = set()
    for role in roles:
        roles_set.add(role)
    roles_list = list(roles_set)
    roles_list.sort()
    return '|'.join(roles_list)
    
data_edited['Artist Role'] = data_edited['Artist Role'].apply(items_to_set)

In [ ]:
print('After: '+str(len(data_edited['Artist Role'].unique())))

### Feature: Artist Nationality

In [ ]:
print('Before: '+str(len(data['Artist Nationality'].unique())))

In [ ]:
data_edited['Artist Nationality'] = data_edited['Artist Nationality'].apply(items_to_set)

In [ ]:
print('After: '+str(len(data_edited['Artist Nationality'].unique())))

## Select some features where you expect integrity problems (describe your choice) and check integrity of those features.

### Feature: Excavation

In [ ]:
def get_date(input):
    dates = re.findall(r'\d+', str(input))
    dates_copy = dates.copy()
    out = ''
    if len(dates)==0:
        return np.NaN
    elif len(dates)==1:
        return dates[0]
    for i in range(len(dates)):
        if i%2==1:
            if len(dates[i])==2:
                dates_copy[i]= dates_copy[i-1][0:2]+dates[i]
            elif len(dates[i])==1:
                dates_copy[i]= dates_copy[i-1][0:3]+dates[i]
            
        if i%2==1:
            out +='-'
            out +=dates_copy[i]
            if not (i+1)== len(dates):
                out+='|'
        else:
            out +=dates_copy[i]
            
    return out      

In [ ]:
data_edited['Excavation date'] = data_edited['Excavation'].apply(get_date)

## Convert at least five features to a proper data type. Choose at least one numeric, one categorical and one datetime.

In [ ]:
data_edited['Is Public Domain'] = data_edited['Is Public Domain'].astype('bool')
data_edited['Is Highlight'] = data_edited['Is Highlight'].astype('bool')
data_edited['Object ID'] = data_edited['Object ID'].astype('int64')
department_cat = pd.api.types.CategoricalDtype(categories= data['Department'].unique(),ordered=False)
data_edited['Department'] = data_edited['Department'].astype(department_cat)
data_edited['Metadata Date'] =  pd.to_datetime(data_edited['Metadata Date'], format='%m/%d/%Y %I:%M:%S %p')

## Find some outliers (describe your selection criteria).

In [ ]:
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR =  Q3 - Q1

DateOut = (data['Object End Date'] > Q3['Object End Date'] + 2*IQR['Object End Date']) | (data['Object End Date'] < Q1['Object End Date'] - 2*IQR['Object End Date'])
display(DateOut.sum())

## Detect missing data in at least three features, convert them to a proper representation (if they are already not), and impute missing values in at least one feature.

### Missing data in:
*  Artist Prefix
*  Geography Type
*  Portfolio
*  State

There is no suitable feature, where can I impute missing values.

In [ ]:
data_edited['Artist Prefix'].replace(['?'], np.nan, inplace=True)
data_edited['Artist Prefix'].replace([''], np.nan, inplace=True)

data_edited['Geography Type'].replace(['|'], np.nan, inplace=True)
data_edited['Geography Type'].replace(['||'], np.nan, inplace=True)

data_edited['Portfolio'].replace(['\r\n'], np.nan, inplace=True)

data_edited['State'].replace(['|'], np.nan, inplace=True)
data_edited['State'].replace(['||'], np.nan, inplace=True)

## Focus more precisely on cleaning of the "Medium" feature.

In [ ]:
data_edited['Medium'].replace(['\r\n'], np.nan, inplace=True)
data_edited['Medium'] = data_edited['Medium'].str.replace('\r\n',' ')
data_edited['Medium'] = data_edited['Medium'].str.strip()

## Dimensions cleaning

In [ ]:
def process_dim(input):
    numbers = re.findall('\(.*?\)',str(input))
    out = []
    for item in numbers:
        unit = ''
        subnumbers = re.findall(r"[-+]?\d*\.\d+|\d+", str(item))
        if len(subnumbers)==0:
            continue
        if 'cm' in item:
            unit = 'cm'
        elif 'g' in item:
            unit = 'g'
        for subnumber in subnumbers:
            out.append(subnumber+unit)
    if len(out)==0:
        return np.nan
    return '|'.join(out)
    
data_edited['Dimensions'] = data_edited['Dimensions'].str.replace('\r\n',' ')
data_edited['Dimensions'].replace(['`'], np.nan, inplace=True)
data_edited['Dimensions'].replace([''], np.nan, inplace=True)
data_edited['Dimensions'] = data_edited['Dimensions'].str.strip()
data_edited['Dimensions_cleaned'] = data_edited['Dimensions'].apply(process_dim)
#data_edited['Dimensions_cleaned']